# Running GPU and Clifford simulations

This notebook shows how you can use the QDK to simulate a program with full-state simulators that run on CPU and GPU, as well as a Clifford simulation of programs that use all Clifford-compatible gates.

First, we compile the given OpenQASM source code to generate a QIR program and configure a noise model we'd like to use during simulation.

In [ ]:
from qdk import openqasm, qsharp, widgets
from qdk.simulation import NoiseConfig, run_qir

SHOTS = 100_000

qsharp.init()

src = """
OPENQASM 3.0;
include "stdgates.inc";
qubit[4] q;
h q[0];
rz(1.86) q[0];
h q[0];
sdg q[0];
barrier q;
x q[1];
cx q[0], q[1];
cx q[0], q[2];
cx q[1], q[3];
h q;
bit[4] c = measure q;
"""
qir = openqasm.compile(src)

# Build a noise model
noise = NoiseConfig()
noise.h.loss = 0.005
noise.cx.loss = 0.005
noise.cx.ix = 0.0025
noise.cx.xi = 0.0025

Given the compiled QIR program, we can simulate it using the GPU full-state simulator, passing in the number of shots and the noise model.

In [ ]:
results = run_qir(qir, shots=SHOTS, noise=noise, type="gpu")
widgets.Histogram(results, items = "top-25", sort="high-to-low", labels="kets")

We can also simulate using the CPU full-state simulator, which should produce similar results.

In [ ]:
results = run_qir(qir, shots=SHOTS, noise=noise, type="cpu")
widgets.Histogram(results, items = "top-25", sort="high-to-low", labels="kets")

Finally, if we use a program that has only Clifford compatible gates, we can use the Clifford simulator instead. This style of simulation can scale to much higher number of qubits and shots.

In [ ]:
# Note that the Clifford simulator only accepts Clifford gates.
# So, we round the rz(1.86) rotation to the nearest Clifford gate, an s gate.
qir_clifford = openqasm.compile("""
OPENQASM 3.0;
include "stdgates.inc";
qubit[4] q;
h q[0];
s q[0]; // rounding of rz(1.86)
h q[0];
sdg q[0];
barrier q;
x q[1];
cx q[0], q[1];
cx q[0], q[2];
cx q[1], q[3];
h q;
bit[4] c = measure q;
""")

results = run_qir(qir_clifford, shots=SHOTS, noise=noise, type="clifford")
widgets.Histogram(results, items = "top-25", sort="high-to-low", labels="kets")